# Problem 1

In [2]:
!git clone https://github.com/qfgaohao/pytorch-ssd.git

Cloning into 'pytorch-ssd'...
remote: Enumerating objects: 812, done.
remote: Total 812 (delta 0), reused 0 (delta 0), pack-reused 812
Receiving objects: 100% (812/812), 1.05 MiB | 21.94 MiB/s, done.
Resolving deltas: 100% (544/544), done.


# Part 1

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !tar -xf gdrive/MyDrive/DLS-Homework5/VOCtest_06-Nov-2007.tar -C ./
!tar -xf data/VOCtest_06-Nov-2007.tar -C ./

Mounted at /content/gdrive


Download pretrained pytorch MobilenetV1 SSD and test it locally using Pascal VOC 2007 dataset.

Show the test accuracy for the 20 classes.

In [4]:
!wget -P models https://storage.googleapis.com/models-hao/mobilenet-v1-ssd-mp-0_675.pth
!wget -P models https://storage.googleapis.com/models-hao/voc-model-labels.txt

--2022-05-05 07:39:50--  https://storage.googleapis.com/models-hao/mobilenet-v1-ssd-mp-0_675.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37995286 (36M) [application/octet-stream]
Saving to: ‘models/mobilenet-v1-ssd-mp-0_675.pth’

mobilenet-v1-ssd-mp 100%[===================>]  36.23M  85.1MB/s    in 0.4s    

2022-05-05 07:39:51 (85.1 MB/s) - ‘models/mobilenet-v1-ssd-mp-0_675.pth’ saved [37995286/37995286]

--2022-05-05 07:39:51--  https://storage.googleapis.com/models-hao/voc-model-labels.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 142.250.107.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145 [text/p

### Accuracy

In [5]:
!python pytorch-ssd/eval_ssd.py --net mb1-ssd --dataset VOCdevkit/VOC2007/ --trained_model models/mobilenet-v1-ssd-mp-0_675.pth --label_file models/voc-model-labels.txt

Streaming output truncated to the last 5000 lines.
Load Image: 0.003123 seconds.
Inference time:  0.00634002685546875
Prediction: 0.014458 seconds.
process image 3709
Load Image: 0.003586 seconds.
Inference time:  0.006470441818237305
Prediction: 0.043744 seconds.
process image 3710
Load Image: 0.003584 seconds.
Inference time:  0.0061206817626953125
Prediction: 0.025060 seconds.
process image 3711
Load Image: 0.003651 seconds.
Inference time:  0.006051778793334961
Prediction: 0.038945 seconds.
process image 3712
Load Image: 0.003248 seconds.
Inference time:  0.007855653762817383
Prediction: 0.026135 seconds.
process image 3713
Load Image: 0.003156 seconds.
Inference time:  0.006239652633666992
Prediction: 0.018839 seconds.
process image 3714
Load Image: 0.003259 seconds.
Inference time:  0.006413698196411133
Prediction: 0.024339 seconds.
process image 3715
Load Image: 0.001748 seconds.
Inference time:  0.006092071533203125
Prediction: 0.022150 seconds.
process image 3716
Load Image: 0

# Part 2

In [8]:
!pip install boto3
!python3 pytorch-ssd/open_images_downloader.py --root open_images --class_names "Airplane, Helicopter"

2022-05-05 07:43:18,864 - root - Download https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv.
2022-05-05 07:43:18,939 - root - Download https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv.
2022-05-05 07:43:29,343 - root - Read annotation file open_images/train-annotations-bbox.csv
2022-05-05 07:43:43,769 - root - train bounding boxes size: 24308
2022-05-05 07:43:43,769 - root - Approximate Image Stats: 
2022-05-05 07:43:43,779 - root - Airplane: 12003/14401 = 0.83.
2022-05-05 07:43:43,779 - root - Helicopter: 2398/14401 = 0.17.
2022-05-05 07:43:43,780 - root - Label distribution: 
2022-05-05 07:43:43,781 - root - Airplane: 21285/24308 = 0.88.
2022-05-05 07:43:43,781 - root - Helicopter: 3023/24308 = 0.12.
2022-05-05 07:43:43,781 - root - Shuffle dataset.
2022-05-05 07:43:43,781 - root - Save train data to open_images/sub-train-annotations-bbox.csv.
2022-05-05 07:43:43,973 - root - Download https://storage.googleapis.com/openi

### Training SSD on Airplanes and Helicopters

In [9]:
!python3 pytorch-ssd/train_ssd.py --dataset_type open_images --datasets open_images --net mb1-ssd --pretrained_ssd models/mobilenet-v1-ssd-mp-0_675.pth --scheduler cosine --lr 0.01 --t_max 100 --validation_epochs 5 --num_epochs 100 --base_net_lr 0.001  --batch_size 5

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2022-05-05 07:52:45,510 - root - INFO - Use Cuda.
2022-05-05 07:52:45,510 - root - INFO - Namespace(balance_data=False, base_net=None, base_net_lr=0.001, batch_size=5, checkpoint_folder='models/', dataset_type='open_images', datasets=['open_images'], debug_steps=100, extra_layers_lr=None, freeze_base_net=False, freeze_net=False, gamma=0.1, lr=0.01, mb2_width_mult=1.0, milestones='80,100', momentum=0.9, net='mb1-ssd', num_epochs=100, num_workers=4, pretrained_ssd='models/mobilenet-v1-ssd-mp-0_675.pth', resume=None, scheduler='cosine', t_max=100.0, use_cuda=True, validation_dataset=None, validation_epochs=5, weight_decay=0.0005)
2022-05-05 07:52:45,511 - root - INFO - Prepare training datasets.
2022-05-05 07:52:52,371 - root - INFO - Dataset Summary:Number of Images: 14228
Minimum Number of Images fo

In [10]:
!cp ./models gdrive/MyDrive/DLS-Homework5/models

### Testing without finetunning

In [12]:
!python pytorch-ssd/eval_ssd.py --net mb1-ssd --dataset open_images --trained_model models/mobilenet-v1-ssd-mp-0_675.pth --label_file models/open-images-model-labels.txt

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Traceback (most recent call last):
  File "pytorch-ssd/eval_ssd.py", line 127, in <module>
    class_names = [name.strip() for name in open(args.label_file).readlines()]
FileNotFoundError: [Errno 2] No such file or directory: 'models/open-images-model-labels.txt'


# Part 3

In [ ]:
!pip install onnx

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'pytorch-ssd')

from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd
import torch.onnx
import onnx

model_path = "models/mobilenet-v1-ssd-mp-0_675.pth"
label_path = "models/voc-model-labels.txt"

class_names = [name.strip() for name in open(label_path).readlines()]
num_classes = len(class_names)

net = create_mobilenetv1_ssd(len(class_names), is_test=True)
net.load(model_path)
net = net.to("cuda")
net.eval()
net_type = "mb1-ssd"
model_path = f"models/{net_type}.onnx"
# init_net_path = f"models/{net_type}_init_net.pb"
# init_net_txt_path = f"models/{net_type}_init_net.pbtxt"
# predict_net_path = f"models/{net_type}_predict_net.pb"
# predict_net_txt_path = f"models/{net_type}_predict_net.pbtxt"

dummy_input = torch.randn(1, 3, 300, 300, device="cuda")
torch.onnx.export(net, dummy_input, model_path, verbose=False, output_names=['scores', 'boxes'])

model = onnx.load(model_path)

# Part 4

In [ ]:
!git clone https://github.com/onnx/onnx.git

In [ ]:
!python onnx/onnx/tools/net_drawer.py --input models/mb1-ssd.onnx --output mb1-ssd.dot --embed_docstring
!dot -Tsvg mb1-ssd.dot -o mb1-ssd.svg
!cp mb1-ssd.dot gdrive/MyDrive/DLS-Homework5/mb1-ssd.dot 
!cp mb1-ssd.svg gdrive/MyDrive/DLS-Homework5/mb1-ssd.svg

The stack trace for nodes of different types are: 

TODO: add stack trace from svg

# Part 5

In [ ]:
!docker pull mcr.microsoft.com/onnxruntime/server